In [11]:
!pip install pandas
!pip install pymysql sqlalchemy
!pip install psycopg2-binary

In [12]:
import pandas as pd

north_file_path = "./data/pos_data.csv"
south_file_path = "./data/erp_data.csv"

df_north = pd.read_csv(north_file_path)
df_south = pd.read_csv(south_file_path)

df_north["location"] = "north"
df_south["location"] = "south"

df_north.head(), df_south.head()

(                YNSM_NO   YNSM_CUST             ynsd_prod  ynsd_qty  \
 0  S2530030              A0566       PS05848DG54B046B1G01         1   
 1  S2530030              A0566       PS05848DH68B046B1G01         1   
 2  S2530030              A0566       PS05848DLH4B046B1G01         2   
 3  S2530030              A0566       PS05848DH48B04519G01         2   
 4  S2530030              A0566       PS05848DH48B0451BG01         2   
 
     ynsd_amt    YN_DATE location  
 0  477727.27  3/12/2025    north  
 1  477727.27  3/12/2025    north  
 2  477727.27  3/12/2025    north  
 3  477727.27  3/12/2025    north  
 4  477727.27  3/12/2025    north  ,
           PS_NO         dd cus_no cur_id                 prd_no   qty  \
 0  SA250312F004  3/12/2025  GG047    USD  SDC016040B07000000102   500   
 1  SA250312F004  3/12/2025  GG047    USD  SDC046040B07000000101  1000   
 2  SA250312F004  3/12/2025  GG047    USD  SDC056040B07000000102  1000   
 3  SA250312F004  3/12/2025  GG047    USD  SDC406040B

In [13]:
df_north.rename(columns={"YNSM_NO": "order_no", "YNSM_CUST": "customer_id",
                         "ynsd_prod": "product_id", "ynsd_qty": "quantity",
                         "ynsd_amt": "amount", "YN_DATE": "date"}, inplace=True)

df_south.rename(columns={"PS_NO": "order_no", "cus_no": "customer_id",
                         "prd_no": "product_id", "qty": "quantity",
                         "amt": "amount", "dd": "date"}, inplace=True)

df_north["date"] = pd.to_datetime(df_north["date"], errors='coerce')
df_south["date"] = pd.to_datetime(df_south["date"], errors='coerce')
df_south["cur_id"].fillna("VND", inplace=True)

df_north.head(), df_south.head()

(               order_no customer_id            product_id  quantity  \
 0  S2530030              A0566       PS05848DG54B046B1G01         1   
 1  S2530030              A0566       PS05848DH68B046B1G01         1   
 2  S2530030              A0566       PS05848DLH4B046B1G01         2   
 3  S2530030              A0566       PS05848DH48B04519G01         2   
 4  S2530030              A0566       PS05848DH48B0451BG01         2   
 
       amount       date location  
 0  477727.27 2025-03-12    north  
 1  477727.27 2025-03-12    north  
 2  477727.27 2025-03-12    north  
 3  477727.27 2025-03-12    north  
 4  477727.27 2025-03-12    north  ,
        order_no       date customer_id cur_id             product_id  \
 0  SA250312F004 2025-03-12       GG047    USD  SDC016040B07000000102   
 1  SA250312F004 2025-03-12       GG047    USD  SDC046040B07000000101   
 2  SA250312F004 2025-03-12       GG047    USD  SDC056040B07000000102   
 3  SA250312F004 2025-03-12       GG047    USD  SDC406040

In [14]:
import pandas as pd

exchange_rate_file_path = "./data/USD_VND_HistoricalData.csv"

df_exchange = pd.read_csv(exchange_rate_file_path)

df_exchange["Date"] = pd.to_datetime(df_exchange["Date"], errors='coerce')

df_exchange.rename(columns={"Date": "exchange_date", "Price": "exchange_rate"}, inplace=True)

df_exchange = df_exchange.sort_values(by="exchange_date")

date_range = pd.date_range(start=df_exchange["exchange_date"].min(), end=df_exchange["exchange_date"].max(), freq="D")

df_full = pd.DataFrame(date_range, columns=["exchange_date"])

df_merged = df_full.merge(df_exchange, on="exchange_date", how="left")

df_merged["exchange_rate"] = df_merged["exchange_rate"].ffill()

print(df_merged.tail(20))


     exchange_date exchange_rate      Open      High       Low Vol. Change %
1886    2025-03-01      25,530.0       NaN       NaN       NaN  NaN      NaN
1887    2025-03-02      25,530.0       NaN       NaN       NaN  NaN      NaN
1888    2025-03-03      25,575.0  25,554.0  25,617.5  25,548.0  NaN    0.18%
1889    2025-03-04      25,540.0  25,580.0  25,596.5  25,555.0  NaN   -0.14%
1890    2025-03-05      25,490.0  25,515.0  25,586.0  25,495.0  NaN   -0.20%
1891    2025-03-06      25,485.0  25,450.0  25,516.5  25,430.0  NaN   -0.02%
1892    2025-03-07      25,480.0  25,449.0  25,552.5  25,435.0  NaN   -0.02%
1893    2025-03-08      25,480.0       NaN       NaN       NaN  NaN      NaN
1894    2025-03-09      25,480.0       NaN       NaN       NaN  NaN      NaN
1895    2025-03-10      25,525.0  25,515.0  25,568.5  25,510.0  NaN    0.18%
1896    2025-03-11      25,450.0  25,512.5  25,520.0  25,462.0  NaN   -0.29%
1897    2025-03-12      25,445.0  25,485.0  25,515.0  25,450.0  NaN   -0.02%

In [15]:
df_south = df_south.merge(df_merged[["exchange_date", "exchange_rate"]], left_on="date", right_on="exchange_date", how="left")

df_south.drop(columns=["exchange_date"], inplace=True)

df_south["exchange_rate"] = df_south["exchange_rate"].fillna("0")
df_south["exchange_rate"] = df_south["exchange_rate"].str.replace(",", "").astype(float)

df_south["amount_vnd"] = df_south.apply(
    lambda row: row["amount"] if row["cur_id"] == "VND" else row["amount"] * row["exchange_rate"], axis=1
)

df_south.head()

,order_no,date,customer_id,cur_id,product_id,quantity,amount,location,exchange_rate,amount_vnd
0,SA250312F004,2025-03-12,GG047,USD,SDC016040B07000000102,500,5.0,south,25445.0,127225.0
1,SA250312F004,2025-03-12,GG047,USD,SDC046040B07000000101,1000,70.0,south,25445.0,1781150.0
2,SA250312F004,2025-03-12,GG047,USD,SDC056040B07000000102,1000,10.0,south,25445.0,254450.0
3,SA250312F004,2025-03-12,GG047,USD,SDC406040B07000000402,500,70.0,south,25445.0,1781150.0
4,SA250312F018,2025-03-12,A0379,VND,SXV02529SCA003001,200,6054545.0,south,25445.0,6054545.0


In [16]:
df_north = df_north.merge(df_merged[["exchange_date", "exchange_rate"]], left_on="date", right_on="exchange_date", how="left")

df_north.drop(columns=["exchange_date"], inplace=True)

df_south["exchange_rate"] = df_south["exchange_rate"].fillna("0")
df_north["exchange_rate"] = df_north["exchange_rate"].str.replace(",", "").astype(float)

df_north["cur_id"] = "VND"

df_north["amount_vnd"] = df_north.apply(
    lambda row: row["amount"] if row["cur_id"] == "VND" else row["amount"] * row["exchange_rate"], axis=1
)

df_north.head()

,order_no,customer_id,product_id,quantity,amount,date,location,exchange_rate,cur_id,amount_vnd
0,S2530030,A0566,PS05848DG54B046B1G01,1,477727.27,2025-03-12,north,25445.0,VND,477727.27
1,S2530030,A0566,PS05848DH68B046B1G01,1,477727.27,2025-03-12,north,25445.0,VND,477727.27
2,S2530030,A0566,PS05848DLH4B046B1G01,2,477727.27,2025-03-12,north,25445.0,VND,477727.27
3,S2530030,A0566,PS05848DH48B04519G01,2,477727.27,2025-03-12,north,25445.0,VND,477727.27
4,S2530030,A0566,PS05848DH48B0451BG01,2,477727.27,2025-03-12,north,25445.0,VND,477727.27


In [17]:
client_file_path = "./data/client_data.csv"
product_file_path = "./data/product_data.csv"

df_client = pd.read_csv(client_file_path)
df_product = pd.read_csv(product_file_path)



In [18]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

DB_USER = "dw_user"
DB_PASSWORD = quote_plus("dw_dss_BK@242")
DB_HOST = "localhost"
DB_PORT = "5433"
DB_NAME = "sales_data_warehouse"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

df_all = pd.concat([df_north, df_south], ignore_index=True)

#----------------------------------


In [19]:

# df_all['date'] = pd.to_datetime(df_all['date']).dt.date 
# df_all['location'] = df_all['location'].str.strip() 
# df_all['customer_id'] = df_all['customer_id'].str.strip()
# df_all['product_id'] = df_all['product_id'].str.strip()
# df_all['cur_id'] = df_all['cur_id'].str.strip().str.upper() 


# try:
#     with engine.connect() as connection:
#         with connection.begin():
#             print("Transaction started.")

#             # === 1. Load DIM_AREA ===
#             print("Loading dim_area...")
#             unique_locations = df_all[['location']].drop_duplicates().dropna()
#             if not unique_locations.empty:
#                 area_data = unique_locations.to_dict(orient='records')
#                 stmt_area = text("""
#                     INSERT INTO dim_area (area_name)
#                     VALUES (:location)
#                     ON CONFLICT (area_name) DO NOTHING;
#                 """)
#                 connection.execute(stmt_area, area_data)
#                 print(f"Processed {len(area_data)} unique areas.")

#             # Lấy mapping area_name -> area_id
#             area_map_result = connection.execute(text("SELECT area_name, area_id FROM dim_area;"))
#             area_map = {row.area_name: row.area_id for row in area_map_result}
#             print("Area map created.")

#             # === 2. Load DIM_DATE ===
#             print("Loading dim_date...")
#             unique_dates_df = df_all[['date']].drop_duplicates().dropna()
#             if not unique_dates_df.empty:
#                 date_data = []
#                 for date_val in unique_dates_df['date']:
#                     date_data.append({
#                         'date': date_val,
#                         'year': date_val.year,
#                         'month': date_val.month,
#                         'day': date_val.day,
#                         'quarter': (date_val.month - 1) // 3 + 1
#                     })

#                 stmt_date = text("""
#                     INSERT INTO dim_date (date, year, month, day, quarter)
#                     VALUES (:date, :year, :month, :day, :quarter)
#                     ON CONFLICT (date) DO NOTHING;
#                 """)
#                 connection.execute(stmt_date, date_data)
#                 print(f"Processed {len(date_data)} unique dates.")

#             # Lấy mapping date -> date_id
#             date_map_result = connection.execute(text("SELECT date, date_id FROM dim_date;"))
#             date_map = {row.date: row.date_id for row in date_map_result}
#             print("Date map created.")


#             # === 3. Load DIM_CURRENCY ===
#             print("Loading dim_currency...")
#             unique_currencies = df_all[['cur_id']].drop_duplicates().dropna()
#             if not unique_currencies.empty:
#                 currency_data = unique_currencies.rename(columns={'cur_id': 'cur_code'}).to_dict(orient='records')
#                 stmt_currency = text("""
#                     INSERT INTO dim_currency (cur_code)
#                     VALUES (:cur_code)
#                     ON CONFLICT (cur_code) DO NOTHING;
#                 """)
#                 connection.execute(stmt_currency, currency_data)
#                 print(f"Processed {len(currency_data)} unique currencies.")

#             # Lấy mapping cur_code -> currency_id
#             currency_map_result = connection.execute(text("SELECT cur_code, currency_id FROM dim_currency;"))
#             currency_map = {row.cur_code: row.currency_id for row in currency_map_result}
#             print("Currency map created.")


#             # === 4. Load DIM_CUSTOMER ===
#             print("Loading dim_customer...")
#             unique_customers = df_all[['customer_id', 'location']].drop_duplicates().dropna(subset=['customer_id'])
#             if not unique_customers.empty:
#                 customer_data = []
#                 for _, row in unique_customers.iterrows():
#                     area_id = area_map.get(row['location'])
#                     if area_id is None:
#                         print(f"Warning: Skipping customer {row['customer_id']} due to unknown location '{row['location']}'.")
#                         continue
#                     customer_data.append({
#                         'cus_no': row['customer_id'],
#                         'customer_name': row['customer_id'],
#                         'area_id': area_id
#                     })

#                 if customer_data: 
#                     stmt_customer = text("""
#                         INSERT INTO dim_customer (cus_no, customer_name, area_id)
#                         VALUES (:cus_no, :customer_name, :area_id)
#                         ON CONFLICT (cus_no) DO NOTHING;
#                     """)
#                     connection.execute(stmt_customer, customer_data)
#                     print(f"Processed {len(customer_data)} unique customers.")

#             # Lấy mapping cus_no -> customer_id (PK của bảng)
#             customer_map_result = connection.execute(text("SELECT cus_no, customer_id FROM dim_customer;"))
#             customer_map = {row.cus_no: row.customer_id for row in customer_map_result}
#             print("Customer map created.")


#             # === 5. Load DIM_PRODUCT ===
#             print("Loading dim_product...")
#             unique_products = df_all[['product_id']].drop_duplicates().dropna()
#             if not unique_products.empty:
#                 product_data = []
#                 for _, row in unique_products.iterrows():
#                     # Giả định tên = mã SP và các trường khác là NULL
#                     product_data.append({
#                         'prd_no': row['product_id'],
#                         'prd_name': row['product_id'], # Giả sử tên = mã SP
#                         'logo': None, 'model': None, 'std': None, 'color': None,
#                         'sqa': None, 'sqa_color': None, 'size_': None
#                     })

#                 stmt_product = text("""
#                     INSERT INTO dim_product
#                     (prd_no, prd_name, logo, model, std, color, sqa, sqa_color, size_)
#                     VALUES
#                     (:prd_no, :prd_name, :logo, :model, :std, :color, :sqa, :sqa_color, :size_)
#                     ON CONFLICT (prd_no) DO NOTHING;
#                 """)
#                 connection.execute(stmt_product, product_data)
#                 print(f"Processed {len(product_data)} unique products.")

#             # Lấy mapping prd_no -> product_id (PK của bảng)
#             product_map_result = connection.execute(text("SELECT prd_no, product_id FROM dim_product;"))
#             product_map = {row.prd_no: row.product_id for row in product_map_result}
#             print("Product map created.")


#             # === 6. Load FACT_SALES ===
#             print("Loading fact_sales...")
#             fact_data = []
#             processed_count = 0
#             skipped_count = 0

#             for index, row in df_all.iterrows():
#                 # Lookup foreign keys
#                 date_id = date_map.get(row['date'])
#                 customer_id_fk = customer_map.get(row['customer_id'])
#                 currency_id_fk = currency_map.get(row['cur_id'])
#                 product_id_fk = product_map.get(row['product_id'])

#                 # Validate foreign keys
#                 if None in [date_id, customer_id_fk, currency_id_fk, product_id_fk]:
#                     skipped_count += 1
#                     continue


#                 try:
#                     qty = int(row['quantity'])
#                     amt = float(row['amount'])
#                     amt_vnd = float(row['amount_vnd'])
#                     exchange_rate = float(row['exchange_rate'])
#                     if qty <= 0 or amt < 0 or exchange_rate <= 0:
#                          skipped_count += 1
#                          continue
#                 except (ValueError, TypeError):
#                     skipped_count += 1
#                     continue


#                 fact_data.append({
#                     'ps_no': row['order_no'],
#                     'date_id': date_id,
#                     'customer_id': customer_id_fk,
#                     'currency_id': currency_id_fk,
#                     'product_id': product_id_fk,
#                     'qty': qty,
#                     'amt': amt,
#                     'exchange_rate': exchange_rate,
#                     'amt_vnd': amt_vnd
#                 })
#                 processed_count += 1


#             if fact_data:
#                 stmt_fact = text("""
#                     INSERT INTO fact_sales
#                     (ps_no, date_id, customer_id, currency_id, product_id, qty, amt, exchange_rate, amt_vnd)
#                     VALUES
#                     (:ps_no, :date_id, :customer_id, :currency_id, :product_id, :qty, :amt, :exchange_rate, :amt_vnd);
#                 """)

#                 connection.execute(stmt_fact, fact_data)
#                 print(f"Inserted {len(fact_data)} rows into fact_sales.")
#             else:
#                 print("No valid data to insert into fact_sales.")

#             if skipped_count > 0:
#                  print(f"Total rows skipped due to missing FKs or invalid data: {skipped_count}")

#         print("Transaction committed successfully.")

# except Exception as e:
#     print(f"ETL failed. Transaction rolled back. Error: {e}")

# finally:
#     print("ETL process finished.")

In [20]:
import pandas as pd
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# --- Data Cleaning ---
logging.info("Starting data cleaning...")
try:
    # Clean df_all
    df_all['date'] = pd.to_datetime(df_all['date']).dt.date
    df_all['customer_id'] = df_all['customer_id'].astype(str).str.strip()
    df_all['product_id'] = df_all['product_id'].astype(str).str.strip()
    df_all['cur_id'] = df_all['cur_id'].astype(str).str.strip().str.upper()
    df_all['order_no'] = df_all['order_no'].astype(str).str.strip()
    for col in ['quantity', 'amount', 'amount_vnd', 'exchange_rate']:
        df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

    # Clean df_client
    df_client['cus_no'] = df_client['cus_no'].astype(str).str.strip()
    df_client['cus_name'] = df_client['cus_name'].astype(str).str.strip()
    df_client['area_name'] = df_client['area_name'].astype(str).str.strip()

    # Clean df_product
    df_product['prd_no'] = df_product['prd_no'].astype(str).str.strip()
    df_product['prd_name'] = df_product['prd_name'].astype(str).str.strip()
    if 'size' in df_product.columns:
         df_product.rename(columns={'size': 'size_'}, inplace=True)
    for col in ['logo', 'model', 'std', 'color', 'sqa', 'sqa_color', 'size_']:
        if col in df_product.columns:
            if pd.api.types.is_string_dtype(df_product[col]):
                 df_product[col] = df_product[col].str.strip()
            else:
                 df_product[col] = df_product[col].astype(str).str.strip()

    logging.info("Data cleaning finished.")

except Exception as e:
    logging.error(f"Error during data cleaning: {e}", exc_info=True)
    raise

# --- ETL Process ---
try:
    with engine.connect() as connection:
        with connection.begin(): # Start transaction
            logging.info("Transaction started.")

            # === 1. Load DIM_AREA ===

            logging.info("Loading dim_area from df_client...")
            unique_areas = df_client[['area_name']].drop_duplicates().dropna()
            if not unique_areas.empty:
                area_data = unique_areas.to_dict(orient='records')
                stmt_area = text("""
                    INSERT INTO dim_area (area_name)
                    VALUES (:area_name)
                    ON CONFLICT (area_name) DO NOTHING;
                """)
                result = connection.execute(stmt_area, area_data)
                logging.info(f"Processed {len(area_data)} unique areas. Inserted/updated: {result.rowcount}")
            else:
                logging.warning("No unique areas found in df_client to load.")

            area_map_result = connection.execute(text("SELECT area_name, area_id FROM dim_area;"))
            area_map = {row.area_name: row.area_id for row in area_map_result}
            logging.info(f"Area map created with {len(area_map)} entries.")
            if not area_map:
                 logging.warning("Area map is empty. Subsequent customer loading might fail.")


            # === 2. Load DIM_DATE ===

            logging.info("Loading dim_date from df_all...")
            unique_dates_df = df_all[['date']].drop_duplicates().dropna()
            if not unique_dates_df.empty:
                date_data = []
                for date_val in unique_dates_df['date']:
                    if pd.isna(date_val): continue
                    date_data.append({
                        'date': date_val,
                        'year': date_val.year,
                        'month': date_val.month,
                        'day': date_val.day,
                        'quarter': (date_val.month - 1) // 3 + 1
                    })
                if date_data:
                    stmt_date = text("""
                        INSERT INTO dim_date (date, year, month, day, quarter)
                        VALUES (:date, :year, :month, :day, :quarter)
                        ON CONFLICT (date) DO NOTHING;
                    """)
                    result = connection.execute(stmt_date, date_data)
                    logging.info(f"Processed {len(date_data)} unique dates. Inserted/updated: {result.rowcount}")
                else:
                    logging.warning("No valid unique dates found in df_all to load.")
            else:
                 logging.warning("No unique dates found in df_all to load.")

            date_map_result = connection.execute(text("SELECT date, date_id FROM dim_date;"))
            date_map = {pd.to_datetime(row.date).date(): row.date_id for row in date_map_result}
            logging.info(f"Date map created with {len(date_map)} entries.")
            if not date_map:
                 logging.warning("Date map is empty. Subsequent fact loading might fail.")


            # === 3. Load DIM_CURRENCY ===

            logging.info("Loading dim_currency from df_all...")
            unique_currencies = df_all[['cur_id']].drop_duplicates().dropna()
            if not unique_currencies.empty:
                currency_data = unique_currencies.rename(columns={'cur_id': 'cur_code'}).to_dict(orient='records')
                stmt_currency = text("""
                    INSERT INTO dim_currency (cur_code)
                    VALUES (:cur_code)
                    ON CONFLICT (cur_code) DO NOTHING;
                """)
                result = connection.execute(stmt_currency, currency_data)
                logging.info(f"Processed {len(currency_data)} unique currencies. Inserted/updated: {result.rowcount}")
            else:
                 logging.warning("No unique currencies found in df_all to load.")

            currency_map_result = connection.execute(text("SELECT cur_code, currency_id FROM dim_currency;"))
            currency_map = {row.cur_code: row.currency_id for row in currency_map_result}
            logging.info(f"Currency map created with {len(currency_map)} entries.")
            if not currency_map:
                 logging.warning("Currency map is empty. Subsequent fact loading might fail.")


            # === 4. Load DIM_CUSTOMER (Pre-load from df_client) ===

            logging.info("Loading dim_customer from df_client...")
            unique_customers = df_client[['cus_no', 'cus_name', 'area_name']].drop_duplicates(subset=['cus_no']).dropna(subset=['cus_no'])
            if not unique_customers.empty:
                customer_data = []
                for _, row in unique_customers.iterrows():
                    area_id = area_map.get(row['area_name'])
                    if area_id is None:
                        logging.warning(f"Customer '{row['cus_no']}' from df_client has unknown area_name '{row['area_name']}'. Will insert with NULL area_id if needed by facts.")
                    customer_data.append({
                        'cus_no': row['cus_no'],
                        'customer_name': row['cus_name'],
                        'area_id': area_id
                    })
                if customer_data:
                    stmt_customer = text("""
                        INSERT INTO dim_customer (cus_no, customer_name, area_id)
                        VALUES (:cus_no, :customer_name, :area_id)
                        ON CONFLICT (cus_no) DO UPDATE SET
                            customer_name = EXCLUDED.customer_name,
                            area_id = COALESCE(EXCLUDED.area_id, dim_customer.area_id);
                    """)
                    result = connection.execute(stmt_customer, customer_data)
                    logging.info(f"Processed {len(customer_data)} unique customers from df_client. Inserted/updated: {result.rowcount}")
                else:
                     logging.warning("No valid customer data from df_client to pre-load.")
            else:
                logging.warning("No unique customers found in df_client to pre-load.")

            customer_map_result = connection.execute(text("SELECT cus_no, customer_id FROM dim_customer;"))
            customer_map = {row.cus_no: row.customer_id for row in customer_map_result}
            logging.info(f"Customer map created/updated with {len(customer_map)} entries.")


            # === 5. Load DIM_PRODUCT (Pre-load from df_product) ===

            logging.info("Loading dim_product from df_product...")
            required_cols = ['prd_no', 'prd_name', 'logo', 'model', 'std', 'color', 'sqa', 'sqa_color', 'size_']
            cols_to_use = [col for col in required_cols if col in df_product.columns]
            unique_products = df_product[cols_to_use].drop_duplicates(subset=['prd_no']).dropna(subset=['prd_no'])
            if not unique_products.empty:
                product_data = []
                for _, row in unique_products.iterrows():
                    data_dict = {'prd_no': row['prd_no']}
                    for col in required_cols[1:]:
                         data_dict[col] = row.get(col, None)
                    product_data.append(data_dict)
                if product_data:
                    stmt_product = text(f"""
                        INSERT INTO dim_product
                        (prd_no, prd_name, logo, model, std, color, sqa, sqa_color, size_)
                        VALUES
                        (:prd_no, :prd_name, :logo, :model, :std, :color, :sqa, :sqa_color, :size_)
                        ON CONFLICT (prd_no) DO UPDATE SET
                            prd_name = EXCLUDED.prd_name, logo = EXCLUDED.logo, model = EXCLUDED.model,
                            std = EXCLUDED.std, color = EXCLUDED.color, sqa = EXCLUDED.sqa,
                            sqa_color = EXCLUDED.sqa_color, size_ = EXCLUDED.size_;
                    """)
                    result = connection.execute(stmt_product, product_data)
                    logging.info(f"Processed {len(product_data)} unique products from df_product. Inserted/updated: {result.rowcount}")
                else:
                    logging.warning("No valid product data from df_product to pre-load.")
            else:
                logging.warning("No unique products found in df_product to pre-load.")

            product_map_result = connection.execute(text("SELECT prd_no, product_id FROM dim_product;"))
            product_map = {row.prd_no: row.product_id for row in product_map_result}
            logging.info(f"Product map created/updated with {len(product_map)} entries.")



            # === 6. Load FACT_SALES ===
            logging.info("Loading fact_sales from df_all...")
            fact_data = []
            processed_count = 0
            skipped_count = 0
            invalid_value_count = 0
            missing_fk_count = 0 # Đếm số dòng bị skip do FK thiếu (date, currency, customer, product)
            customer_created_count = 0
            product_created_count = 0 # Biến đếm sản phẩm được tạo mới

            # --- Câu lệnh tạo mới và lấy ID (định nghĩa ngoài vòng lặp) ---
            stmt_new_customer = text("""
                INSERT INTO dim_customer (cus_no, customer_name, area_id)
                VALUES (:cus_no, :customer_name, :area_id)
                ON CONFLICT (cus_no) DO NOTHING;
            """)
            stmt_get_customer_id = text("SELECT customer_id FROM dim_customer WHERE cus_no = :cus_no")

            stmt_new_product = text("""
                INSERT INTO dim_product (prd_no, prd_name, logo, model, std, color, sqa, sqa_color, size_)
                VALUES (:prd_no, :prd_name, :logo, :model, :std, :color, :sqa, :sqa_color, :size_)
                ON CONFLICT (prd_no) DO NOTHING;
            """)
            stmt_get_product_id = text("SELECT product_id FROM dim_product WHERE prd_no = :prd_no")



            for index, row in df_all.iterrows():

                customer_cus_no = row.get('customer_id')
                product_prd_no = row.get('product_id')
                date_val = row.get('date')
                cur_code = row.get('cur_id')
                order_no_val = row.get('order_no')

                if pd.isna(customer_cus_no) or pd.isna(product_prd_no) or pd.isna(date_val) or pd.isna(cur_code) or pd.isna(order_no_val):
                    logging.warning(f"Skipping row {index}: Missing key information in source data.")
                    skipped_count += 1
                    continue

                # --- Tra cứu FKs ---
                date_id = date_map.get(date_val)
                customer_id_fk = customer_map.get(customer_cus_no)
                currency_id_fk = currency_map.get(cur_code)
                product_id_fk = product_map.get(product_prd_no) # Tra cứu sản phẩm

                # --- Xử lý khách hàng thiếu ---
                if customer_id_fk is None:

                    logging.warning(f"Fact row {index}: Customer '{customer_cus_no}' not found. Attempting to create.")
                    try:
                        connection.execute(stmt_new_customer, {'cus_no': customer_cus_no, 'customer_name': customer_cus_no, 'area_id': None})
                        result_cus_id = connection.execute(stmt_get_customer_id, {'cus_no': customer_cus_no}).fetchone()
                        if result_cus_id:
                            customer_id_fk = result_cus_id.customer_id
                            customer_map[customer_cus_no] = customer_id_fk
                            customer_created_count += 1
                            logging.info(f"Fact row {index}: Successfully created/retrieved customer_id {customer_id_fk} for cus_no '{customer_cus_no}'.")
                        else:
                            logging.error(f"Fact row {index}: Failed to retrieve customer_id for '{customer_cus_no}' after attempting insert. Skipping row.")
                            missing_fk_count += 1; skipped_count += 1; continue
                    except Exception as insert_exc:
                        logging.error(f"Fact row {index}: Error processing missing customer '{customer_cus_no}': {insert_exc}. Skipping row.", exc_info=True)
                        missing_fk_count += 1; skipped_count += 1; continue

                # --- Xử lý SẢN PHẨM thiếu ---
                if product_id_fk is None:
                    logging.warning(f"Fact row {index}: Product '{product_prd_no}' not found. Attempting to create.")
                    try:
                        # Cố gắng insert SP mới, các thuộc tính khác là None
                        connection.execute(stmt_new_product, {
                            'prd_no': product_prd_no,
                            'prd_name': product_prd_no, # Dùng prd_no làm tên
                            'logo': None, 'model': None, 'std': None, 'color': None,
                            'sqa': None, 'sqa_color': None, 'size_': None
                        })

                        # Lấy product_id (PK) vừa tạo hoặc đã có
                        result_prd_id = connection.execute(stmt_get_product_id, {'prd_no': product_prd_no}).fetchone()

                        if result_prd_id:
                            product_id_fk = result_prd_id.product_id
                            product_map[product_prd_no] = product_id_fk # Cập nhật map cache
                            product_created_count += 1
                            logging.info(f"Fact row {index}: Successfully created/retrieved product_id {product_id_fk} for prd_no '{product_prd_no}'.")
                        else:

                            logging.error(f"Fact row {index}: Failed to retrieve product_id for '{product_prd_no}' after attempting insert. Skipping row.")
                            missing_fk_count += 1
                            skipped_count += 1
                            continue #

                    except Exception as insert_exc:
                        logging.error(f"Fact row {index}: Error processing missing product '{product_prd_no}': {insert_exc}. Skipping row.", exc_info=True)
                        missing_fk_count += 1
                        skipped_count += 1
                        continue 
              


                # --- Kiểm tra TẤT CẢ FKs sau khi đã xử lý KH và SP ---
                #  customer_id_fk và product_id_fk phải có giá trị nếu không có lỗi ở các bước trên
                if None in [date_id, customer_id_fk, currency_id_fk, product_id_fk]:
                    missing_keys_details = []
                    if date_id is None: missing_keys_details.append(f"date: {date_val}")

                    if currency_id_fk is None: missing_keys_details.append(f"currency: {cur_code}")
                    logging.warning(f"Skipping row {index}: Missing other required FK(s) - {', '.join(missing_keys_details)}")
                    missing_fk_count += 1
                    skipped_count += 1
                    continue

                # --- Validate dữ liệu số và các ràng buộc ---
                try:
                    ps_no = str(order_no_val).strip()
                    qty = int(row['quantity'])
                    amt = float(row['amount'])
                    amt_vnd = float(row['amount_vnd'])
                    exchange_rate = float(row['exchange_rate'])

                    if pd.isna(qty) or pd.isna(amt) or pd.isna(amt_vnd) or pd.isna(exchange_rate):
                         raise ValueError("Numeric column contains NaN after conversion.")
                    if not ps_no: raise ValueError("ps_no (order_no) cannot be empty")
                    if qty <= 0: raise ValueError(f"Quantity must be > 0, got {qty}")
                    if amt < 0: raise ValueError(f"Amount must be >= 0, got {amt}")
                    if amt_vnd < 0: raise ValueError(f"Amount VND must be >= 0, got {amt_vnd}")
                    if exchange_rate <= 0: raise ValueError(f"Exchange rate must be > 0, got {exchange_rate}")

                except (ValueError, TypeError, KeyError) as e:
                    logging.warning(f"Skipping row {index} for ps_no '{ps_no}': Invalid data - {e}")
                    invalid_value_count += 1
                    skipped_count += 1
                    continue

                # --- Thêm dữ liệu hợp lệ vào batch ---
                fact_data.append({
                    'ps_no': ps_no,
                    'date_id': date_id,
                    'customer_id': customer_id_fk,
                    'currency_id': currency_id_fk,
                    'product_id': product_id_fk, 
                    'qty': qty,
                    'amt': amt,
                    'exchange_rate': exchange_rate,
                    'amt_vnd': amt_vnd
                })
                processed_count += 1

                



            if fact_data:

                logging.info(f"Inserting final batch of {len(fact_data)} rows into fact_sales...")
                stmt_fact = text("""
                    INSERT INTO fact_sales
                    (ps_no, date_id, customer_id, currency_id, product_id, qty, amt, exchange_rate, amt_vnd)
                    VALUES
                    (:ps_no, :date_id, :customer_id, :currency_id, :product_id, :qty, :amt, :exchange_rate, :amt_vnd);
                """)
                result = connection.execute(stmt_fact, fact_data)
                logging.info(f"Inserted final {result.rowcount} rows into fact_sales.")

            else:
                logging.info("No valid data to insert into fact_sales after processing df_all.")


            logging.info(f"Fact Sales Loading Summary: Processed={processed_count}, Skipped={skipped_count} "
                         f"(Missing Other FKs={missing_fk_count}, Invalid Values={invalid_value_count}), "
                         f"Customers Created approx={customer_created_count}, Products Created approx={product_created_count}")

        logging.info("Transaction committed successfully.")

except Exception as e:
    logging.error(f"ETL failed. Transaction rolled back. Error: {e}", exc_info=True)

finally:
    logging.info("ETL process finished.")
    if 'engine' in locals() and engine:
        engine.dispose()
        logging.info("Database engine disposed.")

2025-04-13 13:46:56,386 - INFO - Starting data cleaning...
2025-04-13 13:46:57,060 - INFO - Data cleaning finished.
2025-04-13 13:46:57,187 - INFO - Transaction started.
2025-04-13 13:46:57,190 - INFO - Loading dim_area from df_client...
2025-04-13 13:46:58,263 - INFO - Processed 45 unique areas. Inserted/updated: 45
2025-04-13 13:46:58,279 - INFO - Area map created with 45 entries.
2025-04-13 13:46:58,280 - INFO - Loading dim_date from df_all...
2025-04-13 13:47:03,159 - INFO - Processed 1553 unique dates. Inserted/updated: 1553
2025-04-13 13:47:03,329 - INFO - Date map created with 1553 entries.
2025-04-13 13:47:03,331 - INFO - Loading dim_currency from df_all...
2025-04-13 13:47:03,390 - INFO - Processed 2 unique currencies. Inserted/updated: 2
2025-04-13 13:47:03,396 - INFO - Currency map created with 2 entries.
2025-04-13 13:47:03,398 - INFO - Loading dim_customer from df_client...
2025-04-13 13:47:04,554 - INFO - Processed 335 unique customers from df_client. Inserted/updated: 33